In [1]:
import pandas as pd

In [4]:
def get_log_days(day):
    day=day.drop_duplicates()
    return day.count()

def is_Wednes(day):
    if day==7 | day==14 | day==21 | day==28:

def feature_process(agg_data_path,log_data_path,data_name):
    agg_data=pd.read_csv(agg_data_path,sep='\t')
    agg_data=pd.get_dummies(agg_data,prefix=['V2', 'V3', 'V4', 'V5'],columns=['V2', 'V3', 'V4', 'V5'])
    #================================================================================================
    log_data=pd.read_csv(log_data_path,sep='\t')
    # 日期
    log_data['OCC_TIM']=log_data['OCC_TIM'].apply(lambda x:pd.to_datetime(x))
    log_data['DAY']=log_data['OCC_TIM'].apply(lambda x:x.day).astype('int')    
    # log_data['DAY'].astype('int')
    log_data['HOUR']=log_data['OCC_TIM'].apply(lambda x:x.hour).astype('int')
    # log_data['HOUR'].astype('int')
    #================================================================================================
    # 用户历史的特征
    log_feature=pd.DataFrame()
    log_feature['USRID']=log_data.sort_values(by='USRID')['USRID'].drop_duplicates()
    # 用户最近有点击行为的日期
    log_feature['lastest_day']=log_data.groupby('USRID')['DAY'].max().tolist()
    # 用户最早有点击行为的日期
    log_feature['earlyest_day']=log_data.groupby('USRID')['DAY'].min().tolist()
    # 用户最近有点击行为与最早有点击行为的日期差
    log_feature['earlyest_lastest_diff']=log_feature['lastest_day']-log_feature['earlyest_day']
    # 用户有点击行为总次数
    log_feature['log_times']=log_data.groupby('USRID')['DAY'].count().tolist()
    # 用户有点击行为的总天数
    log_feature['log_days']=log_data.groupby('USRID')['DAY'].apply(get_log_days).tolist()
    # 是否周三
    # log_feature['is_wen']=log_data['DAY'].apply(lambda x: is_Wednes)
    #================================================================================================
    # 时间差特征
    # 点击天数间隔均值和方差
    pos = log_data.duplicated(subset=['USRID', 'DAY'], keep='first')
    tmp = log_data.loc[~pos, ['USRID', 'DAY']]
    tmp.sort_values(by='DAY', inplace=True)
    mean_var = []
    for i in tmp['USRID'].unique():
        tmp1 = tmp[tmp['USRID'] == i].diff()
        a = tmp1['DAY'].mean()
        b = tmp1['DAY'].var()
        mean_var.append([i, a, b])
    tmp1 = pd.DataFrame(mean_var)
    tmp1.columns = ['USRID', 'day_diff_mean', 'day_diff_var']
    log_feature = log_feature.merge(tmp1, on='USRID',how='left')
    del tmp1, mean_var
    #================================================================================================
    # 小时均值特征
    grouped_hour = log_data.groupby('USRID')['HOUR'].mean().reset_index()
    grouped_hour.columns = ['USRID', 'user_mean_hour']
    grouped_hour_median = log_data.groupby('USRID')['HOUR'].median().reset_index()
    grouped_hour_median.columns = ['USRID', 'user_median_hour']
    grouped_hour_std = log_data.groupby('USRID')['HOUR'].std().reset_index()
    grouped_hour_std.columns = ['USRID', 'user_std_hour']
    grouped_hour_var = log_data.groupby('USRID')['HOUR'].var().reset_index()
    grouped_hour_var.columns = ['USRID', 'user_var_hour']
    # 天均值特征
    grouped_day = log_data.groupby('USRID')['DAY'].mean().reset_index()
    grouped_day.columns = ['USRID', 'user_mean_day']
    grouped_day_median = log_data.groupby('USRID')['DAY'].median().reset_index()
    grouped_day_median.columns = ['USRID', 'user_median_day']
    grouped_day_std = log_data.groupby('USRID')['DAY'].std().reset_index()
    grouped_day_std.columns = ['USRID', 'user_std_day']
    grouped_day_var = log_data.groupby('USRID')['DAY'].var().reset_index()
    grouped_day_var.columns = ['USRID', 'user_var_day']
    # 天小时均值特征
    #  grouped_day_hour = log_data.groupby(['USRID', 'DAY'])['HOUR'].mean().reset_index()
    #  grouped_day_hour.columns = ['USRID', 'DAY', 'user_mean_day_hour']
    # 天小时方差特征
    # grouped_day_hour_var = log_data.groupby(['USRID', 'DAY'])['HOUR'].var().reset_index()
    # grouped_day_hour_var.columns = ['USRID', 'DAY', 'user_mean_day_var']
    
    log_feature=log_feature.merge(grouped_hour,on='USRID',how='left')
    log_feature=log_feature.merge(grouped_hour_median,on='USRID',how='left')
    log_feature=log_feature.merge(grouped_hour_std,on='USRID',how='left')
    log_feature=log_feature.merge(grouped_hour_var,on='USRID',how='left')
    log_feature=log_feature.merge(grouped_day,on='USRID',how='left')
    log_feature=log_feature.merge(grouped_day_median,on='USRID',how='left')
    log_feature=log_feature.merge(grouped_day_std,on='USRID',how='left')
    log_feature=log_feature.merge(grouped_day_var,on='USRID',how='left')
    # log_data=log_data.merge(grouped_day_hour,on=['USRID','DAY'],how='left')
    # log_data=log_data.merge(grouped_day_hour_var,on=['USRID','DAY'],how='left')
    #================================================================================================
    log_data_TCH_TYP_0=log_data[log_data.TCH_TYP==0]
    log_data_TCH_TYP_2=log_data[log_data.TCH_TYP==2]
    
    tch_type0_feature=pd.DataFrame()
    tch_type0_feature['USRID']=log_data_TCH_TYP_0.sort_values(by='USRID')['USRID'].drop_duplicates()
    tch_type0_feature['TCH_TYP_0_num']=log_data_TCH_TYP_0.groupby('USRID')['TCH_TYP'].count().tolist()
    tch_type0_feature['TCH_TYP_0_ratio']=tch_type0_feature['TCH_TYP_0_num']/len(log_data)
    #================================================================================================
    # 小时均值特征
    grouped_hour = log_data_TCH_TYP_0.groupby('USRID')['HOUR'].mean().reset_index()
    grouped_hour.columns = ['USRID', 'user_mean_hour_0']
    grouped_hour_median = log_data_TCH_TYP_0.groupby('USRID')['HOUR'].median().reset_index()
    grouped_hour_median.columns = ['USRID', 'user_median_hour_0']
    grouped_hour_std = log_data_TCH_TYP_0.groupby('USRID')['HOUR'].std().reset_index()
    grouped_hour_std.columns = ['USRID', 'user_std_hour_0']
    grouped_hour_var = log_data_TCH_TYP_0.groupby('USRID')['HOUR'].var().reset_index()
    grouped_hour_var.columns = ['USRID', 'user_var_hour_0']
    # 天均值特征
    grouped_day = log_data_TCH_TYP_0.groupby('USRID')['DAY'].mean().reset_index()
    grouped_day.columns = ['USRID', 'user_mean_day_0']
    grouped_day_median = log_data_TCH_TYP_0.groupby('USRID')['DAY'].median().reset_index()
    grouped_day_median.columns = ['USRID', 'user_median_day_0']
    grouped_day_std = log_data_TCH_TYP_0.groupby('USRID')['DAY'].std().reset_index()
    grouped_day_std.columns = ['USRID', 'user_std_day_0']
    grouped_day_var = log_data_TCH_TYP_0.groupby('USRID')['DAY'].var().reset_index()
    grouped_day_var.columns = ['USRID', 'user_var_day_0']
    # 天小时均值特征
    # grouped_day_hour = log_data_TCH_TYP_0.groupby(['USRID', 'DAY'])['HOUR'].mean().reset_index()
    # grouped_day_hour.columns = ['USRID', 'DAY', 'user_mean_day_hour_0']
    # 天小时方差特征
    # grouped_day_hour_var = log_data_TCH_TYP_0.groupby(['USRID', 'DAY'])['HOUR'].var().reset_index()
    # grouped_day_hour_var.columns = ['USRID', 'DAY', 'user_mean_day_var_0']
    
    tch_type0_feature=tch_type0_feature.merge(grouped_hour,on='USRID',how='left')
    tch_type0_feature=tch_type0_feature.merge(grouped_hour_median,on='USRID',how='left')
    tch_type0_feature=tch_type0_feature.merge(grouped_hour_std,on='USRID',how='left')
    tch_type0_feature=tch_type0_feature.merge(grouped_hour_var,on='USRID',how='left')
    tch_type0_feature=tch_type0_feature.merge(grouped_day,on='USRID',how='left')
    tch_type0_feature=tch_type0_feature.merge(grouped_day_median,on='USRID',how='left')
    tch_type0_feature=tch_type0_feature.merge(grouped_day_std,on='USRID',how='left')
    tch_type0_feature=tch_type0_feature.merge(grouped_day_var,on='USRID',how='left')
    # log_data=log_data.merge(grouped_day_hour,on=['USRID','DAY'],how='left')
    # log_data=log_data.merge(grouped_day_hour_var,on=['USRID','DAY'],how='left')
    #================================================================================================
    tch_type2_feature=pd.DataFrame()
    tch_type2_feature['USRID']=log_data_TCH_TYP_2.sort_values(by='USRID')['USRID'].drop_duplicates()
    tch_type2_feature['TCH_TYP_2_num']=log_data_TCH_TYP_2.groupby('USRID')['TCH_TYP'].count().tolist()
    tch_type2_feature['TCH_TYP_2_ratio']=tch_type2_feature['TCH_TYP_2_num']/len(log_data)
    #================================================================================================
    # 小时均值特征
    grouped_hour = log_data_TCH_TYP_2.groupby('USRID')['HOUR'].mean().reset_index()
    grouped_hour.columns = ['USRID', 'user_mean_hour_2']
    grouped_hour_median = log_data_TCH_TYP_2.groupby('USRID')['HOUR'].median().reset_index()
    grouped_hour_median.columns = ['USRID', 'user_median_hour_2']
    grouped_hour_std = log_data_TCH_TYP_2.groupby('USRID')['HOUR'].std().reset_index()
    grouped_hour_std.columns = ['USRID', 'user_std_hour_2']
    grouped_hour_var = log_data_TCH_TYP_2.groupby('USRID')['HOUR'].var().reset_index()
    grouped_hour_var.columns = ['USRID', 'user_var_hour_2']
    # 天均值特征
    grouped_day = log_data_TCH_TYP_2.groupby('USRID')['DAY'].mean().reset_index()
    grouped_day.columns = ['USRID', 'user_mean_day_2']
    grouped_day_median = log_data_TCH_TYP_2.groupby('USRID')['DAY'].median().reset_index()
    grouped_day_median.columns = ['USRID', 'user_median_day_2']
    grouped_day_std = log_data_TCH_TYP_2.groupby('USRID')['DAY'].std().reset_index()
    grouped_day_std.columns = ['USRID', 'user_std_day_2']
    grouped_day_var = log_data_TCH_TYP_2.groupby('USRID')['DAY'].var().reset_index()
    grouped_day_var.columns = ['USRID', 'user_var_day_2']
    # 天小时均值特征
    # grouped_day_hour = log_data_TCH_TYP_1.groupby(['USRID', 'DAY'])['HOUR'].mean().reset_index()
    # grouped_day_hour.columns = ['USRID', 'DAY', 'user_mean_day_hour_2']
    # 天小时方差特征
    # grouped_day_hour_var = log_data_TCH_TYP_1.groupby(['USRID', 'DAY'])['HOUR'].var().reset_index()
    # grouped_day_hour_var.columns = ['USRID', 'DAY', 'user_mean_day_var_2']
    
    tch_type2_feature=tch_type2_feature.merge(grouped_hour,on='USRID',how='left')
    tch_type2_feature=tch_type2_feature.merge(grouped_hour_median,on='USRID',how='left')
    tch_type2_feature=tch_type2_feature.merge(grouped_hour_std,on='USRID',how='left')
    tch_type2_feature=tch_type2_feature.merge(grouped_hour_var,on='USRID',how='left')
    tch_type2_feature=tch_type2_feature.merge(grouped_day,on='USRID',how='left')
    tch_type2_feature=tch_type2_feature.merge(grouped_day_median,on='USRID',how='left')
    tch_type2_feature=tch_type2_feature.merge(grouped_day_std,on='USRID',how='left')
    tch_type2_feature=tch_type2_feature.merge(grouped_day_var,on='USRID',how='left')
    # log_data=log_data.merge(grouped_day_hour,on=['USRID','DAY'],how='left')
    # log_data=log_data.merge(grouped_day_hour_var,on=['USRID','DAY'],how='left')
    #================================================================================================
    log_data['evt_lbl_0']=log_data['EVT_LBL'].apply(lambda x:x.split('-')[0]).astype('int32')
    df1=log_data.groupby('USRID')['evt_lbl_0'].count().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_count']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data.groupby('USRID')['evt_lbl_0'].var().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_var']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data.groupby('USRID')['evt_lbl_0'].mean().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_mean']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data.groupby('USRID')['evt_lbl_0'].median().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_median']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data.groupby('USRID')['evt_lbl_0'].std().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_std']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    
    log_data['evt_lbl_1']=log_data['EVT_LBL'].apply(lambda x:x.split('-')[1]).astype('int32')
    df1=log_data.groupby('USRID')['evt_lbl_1'].count().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_count']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data.groupby('USRID')['evt_lbl_1'].var().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_var']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data.groupby('USRID')['evt_lbl_1'].mean().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_mean']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data.groupby('USRID')['evt_lbl_1'].median().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_median']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data.groupby('USRID')['evt_lbl_1'].std().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_std']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    
    log_data['evt_lbl_2']=log_data['EVT_LBL'].apply(lambda x:x.split('-')[2]).astype('int32')
    df1=log_data.groupby('USRID')['evt_lbl_2'].count().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_count']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data.groupby('USRID')['evt_lbl_2'].var().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_var']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data.groupby('USRID')['evt_lbl_2'].mean().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_mean']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data.groupby('USRID')['evt_lbl_2'].median().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_median']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data.groupby('USRID')['evt_lbl_2'].std().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_std']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    
    log_feature['evt_llbl01_ratio']=log_data['evt_lbl_0']/log_data['evt_lbl_1']
    log_feature['evt_llbl12_ratio']=log_data['evt_lbl_1']/log_data['evt_lbl_2']
    log_feature['evt_llbl02_ratio']=log_data['evt_lbl_0']/log_data['evt_lbl_2']
    #================================================================================================
    log_data15=log_data[log_data.DAY>=16]
    log_data15['evt_lbl_0']=log_data15['EVT_LBL'].apply(lambda x:x.split('-')[0]).astype('int32')
    df1=log_data15.groupby('USRID')['evt_lbl_0'].count().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_count_15']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data15.groupby('USRID')['evt_lbl_0'].var().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_var_15']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data15.groupby('USRID')['evt_lbl_0'].mean().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_mean_15']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data15.groupby('USRID')['evt_lbl_0'].median().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_median_15']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data15.groupby('USRID')['evt_lbl_0'].std().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_std_15']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    
    log_data15['evt_lbl_1']=log_data15['EVT_LBL'].apply(lambda x:x.split('-')[1]).astype('int32')
    df1=log_data15.groupby('USRID')['evt_lbl_1'].count().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_count_15']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data15.groupby('USRID')['evt_lbl_1'].var().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_var_15']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data15.groupby('USRID')['evt_lbl_1'].mean().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_mean_15']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data15.groupby('USRID')['evt_lbl_1'].median().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_median_15']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data15.groupby('USRID')['evt_lbl_1'].std().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_std_15']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    
    log_data15['evt_lbl_2']=log_data15['EVT_LBL'].apply(lambda x:x.split('-')[2]).astype('int32')
    df1=log_data15.groupby('USRID')['evt_lbl_2'].count().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_count_15']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data15.groupby('USRID')['evt_lbl_2'].var().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_var_15']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data15.groupby('USRID')['evt_lbl_2'].mean().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_mean_15']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data15.groupby('USRID')['evt_lbl_2'].median().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_median_15']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data15.groupby('USRID')['evt_lbl_2'].std().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_std_15']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    
    log_feature['evt_llbl01_ratio_15']=log_data15['evt_lbl_0']/log_data15['evt_lbl_1']
    log_feature['evt_llbl12_ratio_15']=log_data15['evt_lbl_1']/log_data15['evt_lbl_2']
    log_feature['evt_llbl02_ratio_15']=log_data15['evt_lbl_0']/log_data15['evt_lbl_2']
    #================================================================================================
    log_data10=log_data[log_data.DAY>=21]
    log_data10['evt_lbl_0']=log_data10['EVT_LBL'].apply(lambda x:x.split('-')[0]).astype('int32')
    df1=log_data10.groupby('USRID')['evt_lbl_0'].count().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_count_10']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data10.groupby('USRID')['evt_lbl_0'].var().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_var_10']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data10.groupby('USRID')['evt_lbl_0'].mean().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_mean_10']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data10.groupby('USRID')['evt_lbl_0'].median().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_median_10']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data10.groupby('USRID')['evt_lbl_0'].std().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_std_10']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    
    log_data10['evt_lbl_1']=log_data10['EVT_LBL'].apply(lambda x:x.split('-')[1]).astype('int32')
    df1=log_data10.groupby('USRID')['evt_lbl_1'].count().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_count_10']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data10.groupby('USRID')['evt_lbl_1'].var().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_var_10']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data10.groupby('USRID')['evt_lbl_1'].mean().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_mean_10']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data10.groupby('USRID')['evt_lbl_1'].median().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_median_10']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data10.groupby('USRID')['evt_lbl_1'].std().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_std_10']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    
    log_data10['evt_lbl_2']=log_data10['EVT_LBL'].apply(lambda x:x.split('-')[2]).astype('int32')
    df1=log_data10.groupby('USRID')['evt_lbl_2'].count().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_count_10']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data10.groupby('USRID')['evt_lbl_2'].var().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_var_10']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data10.groupby('USRID')['evt_lbl_2'].mean().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_mean_10']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data10.groupby('USRID')['evt_lbl_2'].median().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_median_10']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data10.groupby('USRID')['evt_lbl_2'].std().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_std_10']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    
    log_feature['evt_llbl01_ratio_10']=log_data10['evt_lbl_0']/log_data10['evt_lbl_1']
    log_feature['evt_llbl12_ratio_10']=log_data10['evt_lbl_1']/log_data10['evt_lbl_2']
    log_feature['evt_llbl02_ratio_10']=log_data10['evt_lbl_0']/log_data10['evt_lbl_2']
    #================================================================================================
    log_data7=log_data[log_data.DAY>=24]
    log_data7['evt_lbl_0']=log_data7['EVT_LBL'].apply(lambda x:x.split('-')[0]).astype('int32')
    df1=log_data7.groupby('USRID')['evt_lbl_0'].count().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_count_7']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data7.groupby('USRID')['evt_lbl_0'].var().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_var_7']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data7.groupby('USRID')['evt_lbl_0'].mean().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_mean_7']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data7.groupby('USRID')['evt_lbl_0'].median().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_median_7']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data7.groupby('USRID')['evt_lbl_0'].std().reset_index()
    df1.columns = ['USRID', 'evt_lbl_0_std_7']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    
    log_data7['evt_lbl_1']=log_data7['EVT_LBL'].apply(lambda x:x.split('-')[1]).astype('int32')
    df1=log_data7.groupby('USRID')['evt_lbl_1'].count().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_count_7']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data7.groupby('USRID')['evt_lbl_1'].var().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_var_7']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data7.groupby('USRID')['evt_lbl_1'].mean().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_mean_7']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data7.groupby('USRID')['evt_lbl_1'].median().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_median_7']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data7.groupby('USRID')['evt_lbl_1'].std().reset_index()
    df1.columns = ['USRID', 'evt_lbl_1_std_7']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    
    log_data7['evt_lbl_2']=log_data7['EVT_LBL'].apply(lambda x:x.split('-')[2]).astype('int32')
    df1=log_data7.groupby('USRID')['evt_lbl_2'].count().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_count_7']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data7.groupby('USRID')['evt_lbl_2'].var().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_var_7']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data7.groupby('USRID')['evt_lbl_2'].mean().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_mean_7']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data7.groupby('USRID')['evt_lbl_2'].median().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_median_7']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    df1=log_data7.groupby('USRID')['evt_lbl_2'].std().reset_index()
    df1.columns = ['USRID', 'evt_lbl_2_std_7']
    log_feature=log_feature.merge(df1,on='USRID',how='left')
    
    log_feature['evt_llbl01_ratio_7']=log_data7['evt_lbl_0']/log_data7['evt_lbl_1']
    log_feature['evt_llbl12_ratio_7']=log_data7['evt_lbl_1']/log_data7['evt_lbl_2']
    log_feature['evt_llbl02_ratio_7']=log_data7['evt_lbl_0']/log_data7['evt_lbl_2']
    #================================================================================================
    merge1=log_feature.merge(tch_type0_feature,on='USRID',how='left')
    log_feature=merge1.merge(tch_type2_feature,on='USRID',how='left')
   
    total_data=agg_data.merge(log_feature,on='USRID',how='left')
    total_data.to_csv('deal_data/'+data_name+'.csv',index=None)
    
    print('特征大小：',total_data.shape)

In [6]:
# agg_data_path='data/train_agg.csv'
# log_data_path='data/train_log.csv'
# data_name='train_data'

agg_data_path='data/test_agg.csv'
log_data_path='data/test_log.csv'
data_name='test_data'

feature_process(agg_data_path,log_data_path,data_name)

d:\python\lib\site-packages\ipykernel_launcher.py:224: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
d:\python\lib\site-packages\ipykernel_launcher.py:241: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
d:\python\lib\site-packages\ipykernel_launcher.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
d:\python\lib\site-pac

特征大小： (20000, 160)


In [7]:
train_flg_data=pd.read_csv('data/train_flg.csv',sep='\t')
train_data=pd.read_csv('deal_data/train_data.csv')
train_data_flg=train_data.merge(train_flg_data,on='USRID',how='inner')
train_data_flg.to_csv('deal_data/train_data_flag.csv',index=None)
train_data_flg

,V1,V6,V7,V8,V9,V10,V11,V12,V13,V14,...,TCH_TYP_2_ratio,user_mean_hour_2,user_median_hour_2,user_std_hour_2,user_var_hour_2,user_mean_day_2,user_median_day_2,user_std_day_2,user_var_day_2,FLAG
0,-1.92554,-1.70618,-0.29641,-0.18761,-0.48351,-0.53051,-0.18703,-0.41609,-0.43194,-0.13843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,-1.92554,-1.70618,0.37601,-0.10030,-0.16694,-0.31897,0.13488,-0.41609,-0.43194,-0.13843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,-1.92554,-1.57176,0.03980,-0.17089,-0.38610,-0.46000,-0.36669,-0.17962,-0.43194,-0.13843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,-1.92554,-1.30291,-0.29641,-0.18761,-0.45916,-0.53051,-0.11165,-0.35697,-0.43194,-0.13843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,-1.92554,-1.16849,-0.29641,-0.18761,-0.50786,-0.60103,-0.36994,-0.53433,-0.43194,-0.13843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,-1.92554,-1.16849,-0.29641,-0.18761,-0.50786,-0.60103,-0.36994,-0.53433,-0.43194,-0.13843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,-1.92554,-1.16849,-0.29641,-0.18761,-0.50786,-0.60103,-0.36994,-0.53433,-0.43194,-0.13843,...,2.829801e-07,17.0,17.0,NaN,NaN,16.0,16.0,NaN,NaN,0
7,-1.92554,-1.16849,-0.29641,-0.18761,-0.50786,-0.60103,-0.36994,-0.53433,-0.43194,-0.13843,...,2.829801e-07,11.0,11.0,NaN,NaN,23.0,23.0,NaN,NaN,0
8,-1.92554,-1.16849,-0.29641,-0.18761,-0.19129,-0.13094,-0.35213,0.64802,-0.43194,-0.13843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,-1.92554,-1.16849,-0.29641,-0.18761,-0.09389,-0.22495,-0.15289,0.35243,-0.33390,-0.13843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
